In [ ]:
'''
Perceptronとは
- 機械学習の元祖
    - https://rightcode.co.jp/blog/information-technology/simple-perceptron
- ディープラーニングの大元となるアルゴリズム
    - https://blog.apar.jp/deep-learning/11979/
   
今回やりたいこと
- Perceptronのロジックの実装
- アヤメの花の種別の分類

参考書籍
- Python機械学習プログラミング 達人データサイエンティストによる理論と実践 
    - https://www.amazon.co.jp/dp/4295010073/ref=cm_sw_r_tw_dp_PGPG2W9HVNX1T0NX23XY
'''

In [ ]:
"""
モジュールのimport
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
"""
データのダウンロード
アヤメの特徴量（各列）
  0: がくの長さ(cm)
  1: がくの幅(cm)
  2: 花弁の長さ(cm)
  3: 花弁の幅(cm)
  4: 名前
種別は、Iris-setosa(0~49行)、Iris-versicolor(50~99行)、Iris-virginica(100~149行)
今回は、99行目までを利用する
"""

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

# 全件表示する
pd.set_option('display.max_rows', None)
print(df)

In [ ]:
"""
データ整形

y=目的変数、X=トレーニング変数
"""

# y: 目的変数 Iris-setosa=-1, Iris-versicolor=1
# [1, 1, 1, ..., -1, -1, -1...]
y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', -1, 1)

# X: トレーニング変数 [がくの長さ, 花弁の長さ]の一覧
# [[5.1, 1.4], [4.9, 1.4], [4.7, 1.3], ...]
X = df.iloc[0:100, [0, 2]].values

print('y\n', y)
print('X\n', X)

In [ ]:
"""
グラフ表示(散布図)
"""

data_count = 50

plt.scatter(X[:data_count, 0], X[:data_count, 1], color='red', marker='o', label='setona')
plt.scatter(X[50:50 + data_count, 0], X[50:50 + data_count, 1], color='blue', marker='x', label='versicolor')
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.legend(loc='upper left')
plt.show()

In [ ]:
"""
Perceptronの処理クラス
fit()で学習し、predict()で予測できるようになる
"""

class Perceptron(object):
    # コンストラクタ
    def __init__(self, eta=0.01, n_iter=10):
        # 学習率 0.0〜1.0
        self.eta = eta
        
        # トレーニング回数
        self.n_iter = n_iter
        
    # トレーニング
    def fit(self, X, y):
        # X: トレーニング変数 今回は、2　x　N
        # y: 目的変数

        # 重み(トレーニング結果)
        self.w_ = np.zeros(1 + X.shape[1])
        
        # トレーニングN回目のエラー回数（誤分類回数）
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                # 予測値が正解していたら0, 外していたら符号 * 2 * 学習率
                update = self.eta * (target - self.predict(xi))
                
                # 重みの更新
                self.w_[1:] += update * xi
                self.w_[0] += update
                
                errors += int(update != 0.0)
            self.errors_.append(errors)

        return self

    # 総入力を計算
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    # 分類の予測
    # 特徴量（X)を元に、一致なら1、不一致なら-1を返却する
    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1) 

In [ ]:
"""
Perceptronの処理実行
"""

ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X, y)
plt.plot(range(1, len(ppn.errors_) + 1), ppn.errors_, marker='o')

In [ ]:
"""
グラフ表示関数
"""

def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
    xx1, xx2 = np.meshgrid(
        np.arange(x1_min, x1_max, resolution),
        np.arange(x2_min, x2_max, resolution)
    )
    Z = classifier.predict(
        np.array([
            xx1.ravel(),
            xx2.ravel(),
        ]).T
    )
    
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(
            x=X[y == cl, 0],
            y=X[y == cl, 1],
            alpha=0.8,c=cmap(idx),
            marker=markers[idx],
            label=cl
        )

In [ ]:
"""
処理結果のグラフ表示
赤背景のデータだとsetosa、青背景のデータだとversicolorとして分類される
"""

plot_decision_regions(X, y, classifier=ppn)
plt.xlabel('sepal length [cm]')
plt.ylabel('petap length [cm]')

plt.legend(loc='upper left')

plt.show()